# Study 9 (features)
## Generating image stimuli

Generates an image of a Zarpie with some feature, based on reference images of Zarpies.

https://platform.openai.com/docs/guides/image-generation?api=responses#edit-images

In [16]:
# import packages
from dotenv import load_dotenv
import os
from openai import OpenAI
import base64

# set up API key, client
load_dotenv("../../.env")
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [17]:
# define helper functions
def create_file(file_path):
  with open(file_path, "rb") as file_content:
    result = client.files.create(
        file=file_content,
        purpose="vision",
    )
    return result.id

def encode_image(file_path):
    with open(file_path, "rb") as f:
        base64_image = base64.b64encode(f.read()).decode("utf-8")
    return base64_image

In [32]:
# define all features
features = [
    # physical
    "have wide feet",
    "have straight hair",
    "have freckles on their feet",
    "can touch their toes",
    "have stripes in their hair",
    "have tattoos",
    "can see in the dark",
    "tan easily",
    "have a flat belly button",
    "have bony elbows",
    "paint their toenails",
    "draw circles on their knees",
    "have short nails",
    "wear tall socks",
    "have orange earwax",
    
    # diet
    "eat rice",
    "eat nuts",
    "eat meat",
    "eat cheese",
    "drink milk",
    "like to eat sweets",
    "like to eat broccoli",
    "like to eat herbs",
    "make bread",
    "farm potatoes",
    "like to drink maple syrup",
    "like to drink tea",
    "like to drink fizzy water",
    "really don't like ice cream",
    "really don't like soup",

    # personality
    "are friendly",
    "are curious",
    "are silly",
    "are smart",
    "are thoughtful",
    "are forgetful",
    "are angry",
    "take care of others",
    "live with their families",
    "can do cartwheels",
    "teach children",
    "like to ride bikes",
    "give good gifts",
    "tell bad jokes",
    "are brave",
]

In [ ]:
# get reference images
reference_images_dir = "/Users/MariannaZhang/Library/CloudStorage/Box-Box/NYU/projects/compgenerics/materials/study 9 features/reference_images/"

base64_images = []
file_ids = []
for file in os.listdir(reference_images_dir):
    # check if file is an image
    if file.endswith(('.png', '.jpg', '.jpeg')): 
        image_path = os.path.join(reference_images_dir, file)
        base64_images.append(encode_image(image_path))
        file_ids.append(create_file(image_path))

# load reference images, default prompt as input content
input_content = []

for base64_image in base64_images:
    input_content.append({
        "type": "input_image",
        "image_url": f"data:image/jpeg;base64,{base64_image}",
    })
for file_id in file_ids:
    input_content.append({
        "type": "input_image",
        "file_id": file_id,
    })
    
input_content.append({
    "type": "input_text", 
    "text": "Using the reference images of Zarpies, generate a image of a Zarpie drinking fizzy water."
})

In [ ]:
# prompt model
response = client.responses.create(
    model="gpt-5",
    input=[
        {
            "role": "user",
            "content": input_content
        }
    ],
    tools=[{"type": "image_generation"}],
)

In [30]:
# save image
image_generation_calls = [
    output
    for output in response.output
    if output.type == "image_generation_call"
]

image_data = [output.result for output in image_generation_calls]

if image_data:
    image_base64 = image_data[0]
    
    with open("generated_images/drinks-fizzy-water.png", "wb") as f:
        f.write(base64.b64decode(image_base64))
else:
    print(response.output.content)

In [ ]:
# # follow up
# response_fwup = client.responses.create(
#     model="gpt-5",
#     previous_response_id=response.id,
#     input="Now remove the grass",
#     tools=[{"type": "image_generation"}],
# )

# image_data_fwup = [
#     output.result
#     for output in response_fwup.output
#     if output.type == "image_generation_call"
# ]

# if image_data_fwup:
#     image_base64 = image_data_fwup[0]
#     with open("generated_images/drinks-fizzy-water-2.png", "wb") as f:
#         f.write(base64.b64decode(image_base64))